# Week 3 Assignment - Christina Morgenstern

This assignment involves the writing of Python code for measuring conservation, calculating substitution matrices and calculating distance matrices.

In [14]:
# import libraries for assignment
import numpy as np
from math import log

### 1. Measuring Conservation

In [9]:
# generate random matrix of 7 rows and 21 columns with DNA alphabet
align = np.random.choice(list('ACTG'), (7,21))
align

array([['C', 'T', 'G', 'A', 'T', 'G', 'T', 'G', 'A', 'C', 'A', 'A', 'T',
        'C', 'T', 'C', 'T', 'C', 'G', 'A', 'C'],
       ['T', 'C', 'G', 'A', 'T', 'G', 'C', 'T', 'G', 'A', 'C', 'C', 'G',
        'A', 'G', 'A', 'A', 'T', 'C', 'T', 'T'],
       ['C', 'C', 'T', 'C', 'C', 'G', 'C', 'C', 'G', 'C', 'G', 'C', 'A',
        'T', 'C', 'A', 'C', 'T', 'C', 'A', 'G'],
       ['C', 'C', 'A', 'T', 'T', 'A', 'A', 'G', 'A', 'G', 'T', 'C', 'T',
        'C', 'A', 'C', 'C', 'G', 'C', 'C', 'A'],
       ['A', 'A', 'A', 'C', 'A', 'G', 'A', 'A', 'A', 'T', 'T', 'T', 'A',
        'G', 'T', 'C', 'G', 'C', 'T', 'T', 'A'],
       ['T', 'G', 'A', 'C', 'G', 'G', 'C', 'G', 'C', 'C', 'A', 'G', 'A',
        'T', 'T', 'G', 'C', 'A', 'C', 'C', 'C'],
       ['C', 'C', 'T', 'C', 'T', 'C', 'T', 'T', 'C', 'G', 'T', 'G', 'A',
        'A', 'T', 'G', 'G', 'A', 'C', 'T', 'C']], dtype='<U1')

In [10]:
# DNA scoring matrix
DNA_1 = {'G': { 'G':1, 'C':0, 'A':0, 'T':0 },
         'C': { 'G':0, 'C':1, 'A':0, 'T':0 },
         'A': { 'G':0, 'C':0, 'A':1, 'T':0 },
         'T': { 'G':0, 'C':0, 'A':0, 'T':1 }}

In [11]:
def profile(alignment):
  """ Function to generate a profile represented as a list of dictionaries.
  Takes in an alignment and returns profile."""

  n = len(alignment[0])
  nSeq = float(len(alignment))
  prof = []
  
  for i in range(n):
    counts = {}
    
    for seq in alignment:
      letter = seq[i]
      if letter == '-':
        continue
    
      counts[letter] = counts.get(letter, 0) + 1
    
    for letter in counts:
      counts[letter] /= nSeq
    
    prof.append(counts)

  return prof

In [12]:
def getConservation(align, simMatrix):
  """Function to calculate conservation of sequences in a multiple alignment.
     Takes as input an alignment and a substitution matrix (simMatrix).
     Returns a score for similarity between sequence elements."""
    
  conservation = []
  prof = profile(align)
  
  for compDict in prof:
    
    items = list(compDict.items())  

    items.sort( key=lambda x: x[1] )
        
    score = 0.0
    
    for resA, compA in items:
      for resB, compB in items:
        score += compA * compB * simMatrix[resA][resB]
 
    bestLetter = items[-1][0]
    maxScore = simMatrix[bestLetter][bestLetter]
   
    score /= maxScore
    conservation.append(score)
  
  return conservation


In [13]:
# call conservation function on random DNA alignment
getConservation(align, DNA_1)

[0.42857142857142855,
 0.3877551020408163,
 0.346938775510204,
 0.42857142857142855,
 0.3877551020408163,
 0.5510204081632654,
 0.346938775510204,
 0.30612244897959184,
 0.346938775510204,
 0.30612244897959184,
 0.30612244897959184,
 0.30612244897959184,
 0.42857142857142855,
 0.26530612244897955,
 0.3877551020408163,
 0.346938775510204,
 0.30612244897959184,
 0.26530612244897955,
 0.5510204081632654,
 0.346938775510204,
 0.30612244897959184]

Calling the getConservation function on the random DNA matrix calculates a conservation (similarity) value for each column of the multiple aligned sequences. Low values indicate low similarity and thus low conservation, high values indicate high similarity and thus high conservation. A value of 1 indicates 100% similarity of all residues in the respective column.

### 2. Calculating substitution matrices

In [16]:
# generate 7x21 matrix of alignments containing amino acids and gap (-) elements
align_aa = np.random.choice(list('GAIVLMFYWHCPKRDEQNST-'), (7,21))
align_aa

array([['S', 'Y', 'T', 'K', 'V', 'I', 'Y', 'M', 'Q', 'M', 'L', 'V', 'H',
        'N', 'D', 'F', 'V', 'A', 'I', 'G', 'T'],
       ['K', 'V', 'D', 'E', 'V', 'D', 'E', 'F', 'Q', 'N', 'S', 'F', 'W',
        'H', 'P', 'P', 'A', 'G', 'P', 'F', 'S'],
       ['G', '-', 'G', 'S', 'R', 'Q', 'D', 'S', 'C', 'C', 'Q', 'D', 'W',
        'L', 'S', 'K', 'V', 'E', 'N', 'M', 'Y'],
       ['M', 'V', 'Y', 'K', 'P', 'S', 'M', 'Q', 'D', 'R', 'N', 'E', 'T',
        'C', 'K', 'K', 'V', 'Y', 'S', 'N', 'I'],
       ['W', 'R', 'E', 'H', 'E', 'Y', 'R', 'F', 'W', 'Q', 'N', 'R', 'F',
        'T', 'H', 'F', 'L', 'M', 'F', '-', 'R'],
       ['E', 'R', 'I', 'K', 'D', 'Y', '-', 'T', 'I', 'N', 'C', 'W', 'V',
        'N', 'D', 'Q', 'K', 'K', 'Y', 'D', 'A'],
       ['K', 'D', 'G', 'P', 'A', 'G', 'K', 'D', 'D', 'W', 'F', 'N', '-',
        'S', 'H', 'T', 'E', 'A', 'M', 'M', 'Y']], dtype='<U1')

In [17]:
# substitution matrix for sequence alignment of proteins
BLOSUM62 = {'A':{'A': 4,'R':-1,'N':-2,'D':-2,'C': 0,'Q':-1,'E':-1,'G': 0,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 0,'W':-3,'Y':-2,'V': 0,'X':0},
            'R':{'A':-1,'R': 5,'N': 0,'D':-2,'C':-3,'Q': 1,'E': 0,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 2,'M':-1,'F':-3,'P':-2,'S':-1,'T':-1,'W':-3,'Y':-2,'V':-3,'X':0},
            'N':{'A':-2,'R': 0,'N': 6,'D': 1,'C':-3,'Q': 0,'E': 0,'G': 0,'H': 1,'I':-3,
                 'L':-3,'K': 0,'M':-2,'F':-3,'P':-2,'S': 1,'T': 0,'W':-4,'Y':-2,'V':-3,'X':0},
            'D':{'A':-2,'R':-2,'N': 1,'D': 6,'C':-3,'Q': 0,'E': 2,'G':-1,'H':-1,'I':-3,
                 'L':-4,'K':-1,'M':-3,'F':-3,'P':-1,'S': 0,'T':-1,'W':-4,'Y':-3,'V':-3,'X':0},
            'C':{'A': 0,'R':-3,'N':-3,'D':-3,'C': 9,'Q':-3,'E':-4,'G':-3,'H':-3,'I':-1,
                 'L':-1,'K':-3,'M':-1,'F':-2,'P':-3,'S':-1,'T':-1,'W':-2,'Y':-2,'V':-1,'X':0},
            'Q':{'A':-1,'R': 1,'N': 0,'D': 0,'C':-3,'Q': 5,'E': 2,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 1,'M': 0,'F':-3,'P':-1,'S': 0,'T':-1,'W':-2,'Y':-1,'V':-2,'X':0},
            'E':{'A':-1,'R': 0,'N': 0,'D': 2,'C':-4,'Q': 2,'E': 5,'G':-2,'H': 0,'I':-3,
                 'L':-3,'K': 1,'M':-2,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'G':{'A': 0,'R':-2,'N': 0,'D':-1,'C':-3,'Q':-2,'E':-2,'G': 6,'H':-2,'I':-4,
                 'L':-4,'K':-2,'M':-3,'F':-3,'P':-2,'S': 0,'T':-2,'W':-2,'Y':-3,'V':-3,'X':0},
            'H':{'A':-2,'R': 0,'N': 1,'D':-1,'C':-3,'Q': 0,'E': 0,'G':-2,'H': 8,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-1,'P':-2,'S':-1,'T':-2,'W':-2,'Y': 2,'V':-3,'X':0},
            'I':{'A':-1,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-3,'E':-3,'G':-4,'H':-3,'I': 4,
                 'L': 2,'K':-3,'M': 1,'F': 0,'P':-3,'S':-2,'T':-1,'W':-3,'Y':-1,'V': 3,'X':0},
            'L':{'A':-1,'R':-2,'N':-3,'D':-4,'C':-1,'Q':-2,'E':-3,'G':-4,'H':-3,'I': 2,
                 'L': 4,'K':-2,'M': 2,'F': 0,'P':-3,'S':-2,'T':-1,'W':-2,'Y':-1,'V': 1,'X':0},
            'K':{'A':-1,'R': 2,'N': 0,'D':-1,'C':-3,'Q': 1,'E': 1,'G':-2,'H':-1,'I':-3,
                 'L':-2,'K': 5,'M':-1,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'M':{'A':-1,'R':-1,'N':-2,'D':-3,'C':-1,'Q': 0,'E':-2,'G':-3,'H':-2,'I': 1,
                 'L': 2,'K':-1,'M': 5,'F': 0,'P':-2,'S':-1,'T':-1,'W':-1,'Y':-1,'V': 1,'X':0},
            'F':{'A':-2,'R':-3,'N':-3,'D':-3,'C':-2,'Q':-3,'E':-3,'G':-3,'H':-1,'I': 0,
                 'L': 0,'K':-3,'M': 0,'F': 6,'P':-4,'S':-2,'T':-2,'W': 1,'Y': 3,'V':-1,'X':0},
            'P':{'A':-1,'R':-2,'N':-2,'D':-1,'C':-3,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-4,'P': 7,'S':-1,'T':-1,'W':-4,'Y':-3,'V':-2,'X':0},
            'S':{'A': 1,'R':-1,'N': 1,'D': 0,'C':-1,'Q': 0,'E': 0,'G': 0,'H':-1,'I':-2,
                 'L':-2,'K': 0,'M':-1,'F':-2,'P':-1,'S': 4,'T': 1,'W':-3,'Y':-2,'V':-2,'X':0},
            'T':{'A': 0,'R':-1,'N': 0,'D':-1,'C':-1,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 5,'W':-2,'Y':-2,'V': 0,'X':0},
            'W':{'A':-3,'R':-3,'N':-4,'D':-4,'C':-2,'Q':-2,'E':-3,'G':-2,'H':-2,'I':-3,
                 'L':-2,'K':-3,'M':-1,'F': 1,'P':-4,'S':-3,'T':-2,'W':11,'Y': 2,'V':-3,'X':0},
            'Y':{'A':-2,'R':-2,'N':-2,'D':-3,'C':-2,'Q':-1,'E':-2,'G':-3,'H': 2,'I':-1,
                 'L':-1,'K':-2,'M':-1,'F': 3,'P':-3,'S':-2,'T':-2,'W': 2,'Y': 7,'V':-1,'X':0},
            'V':{'A': 0,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-2,'E':-2,'G':-3,'H':-3,'I': 3,
                 'L': 1,'K':-2,'M': 1,'F':-1,'P':-2,'S':-2,'T': 0,'W':-3,'Y':-1,'V': 4,'X':0},
            'X':{'A': 0,'R': 0,'N': 0,'D': 0,'C': 0,'Q': 0,'E': 0,'G': 0,'H': 0,'I': 0,
                 'L': 0,'K': 0,'M': 0,'F': 0,'P': 0,'S': 0,'T': 0,'W': 0,'Y': 0,'V': 0,'X':0}}

In [18]:
def calcSubstitutionMatrix(alignments, alphabet, maxVal, smooth=5):
 
  """ Function to calculate substitution matrix.
  Takes as arguments a sequence alignment, a sequence alphabet and a maximum value as well as a smoothing value.
  Returns substitution matrix."""

  matrix = {}
  counts = {}
  
  for letterA in alphabet:
    subDict = {}
    
    for letterB in alphabet:
      subDict[letterB] = 0
  
    matrix[letterA] = subDict
    counts[letterA] = 0
  
  totalRes = 0.0
  totalSub = 0.0

  for align in alignments:
 
    numPos = len(align[0])

    for i in range(numPos):
 
      letters = []
      
      for seq in align:

        letter = seq[i]
        if letter == '-':
          continue
    
        letters.append(letter)

      for letterA in letters:
        counts[letterA] += 1
      
        for letterB in letters:          
          matrix[letterA][letterB] += 1

      numLetters = len(letters)
      totalRes += numLetters    
      totalSub += numLetters * numLetters

  averageComp = {}    
  for letter in alphabet:
    averageComp[letter] = counts[letter]/totalRes      

  maxScore = None
  for resA in alphabet:
    for resB in alphabet:

      expected = averageComp[resA] * averageComp[resB]
      
      if not expected:
        continue

      observed = matrix[resA][resB]
      weight = 1.0 / (1.0+(observed/smooth))

      observed /= totalSub
      observed = weight*expected + (1-weight)*observed

      logOdds = log(observed/expected)
                  
      if (maxScore is None) or (logOdds>maxScore):
        maxScore = logOdds
      
      matrix[resA][resB] = logOdds

  maxScore = abs(maxScore)

  for resA in alphabet:
    for resB in alphabet:
      matrix[resA][resB] = int(maxVal*matrix[resA][resB]/maxScore)

  return matrix

In [19]:
# get amino acid residues as keys from BLOSUM62 matrix
aminoAcids = BLOSUM62.keys()

In [20]:
# print substitution matrix of random list of alignments, list of residue letters and a scaling, maximum value for the matrix
print(calcSubstitutionMatrix([align_aa], aminoAcids,10))

{'A': {'A': 9, 'R': 0, 'N': 0, 'D': 0, 'C': 0, 'Q': 0, 'E': 2, 'G': 0, 'H': 0, 'I': 0, 'L': 0, 'K': 0, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 0, 'Y': 2, 'V': 3, 'X': 0}, 'R': {'A': 0, 'R': 7, 'N': 0, 'D': 0, 'C': 0, 'Q': 0, 'E': 0, 'G': 0, 'H': 0, 'I': 0, 'L': 0, 'K': 0, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 0, 'Y': 2, 'V': 4, 'X': 0}, 'N': {'A': 0, 'R': 0, 'N': 8, 'D': -1, 'C': 6, 'Q': 0, 'E': 0, 'G': 0, 'H': 0, 'I': 0, 'L': 4, 'K': 0, 'M': 1, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 0, 'Y': 0, 'V': 0, 'X': 0}, 'D': {'A': 0, 'R': 0, 'N': -1, 'D': 4, 'C': 0, 'Q': 1, 'E': 0, 'G': 0, 'H': 1, 'I': 1, 'L': 0, 'K': -1, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 0, 'Y': 0, 'V': 0, 'X': 0}, 'C': {'A': 0, 'R': 0, 'N': 6, 'D': 0, 'C': 7, 'Q': 4, 'E': 0, 'G': 0, 'H': 0, 'I': 0, 'L': 3, 'K': 0, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 1, 'Y': 0, 'V': 0, 'X': 0}, 'Q': {'A': 0, 'R': 0, 'N': 0, 'D': 1, 'C': 4, 'Q': 7, 'E': 0, 'G': 0, 'H': 0, 'I': 1, 'L': 0, 'K': 0, 'M': 0, 'F': 2, 'P'

### 3. Calculating distance matrices

In [26]:
def calcSeqSimilarity(seqA, seqB, simMatrix):

  """Function to calculate sequence similarity.
  Takes as arguments two sequences and a substitution matrix. 
  Returns similarity score."""
    
  numPlaces = min(len(seqA), len(seqB))
  
  totalScore = 0.0
  
  for i in range(numPlaces):
    
    residueA = seqA[i]
    residueB = seqB[i]
  
    totalScore += simMatrix[residueA][residueB]

  return totalScore


In [27]:
def getDistanceMatrix(seqs, simMatrix):
  
  """Function to generate distance matrix.
  Takes as arguments a list of sequences and a substitution matrix.
  Returns matrix of similarity scores."""

  n = len(seqs)
  matrix = [[0.0] * n for x in range(n)]
  maxScores = [calcSeqSimilarity(x, x, simMatrix) for x in seqs]

  for i in range(n-1):
    seqA = seqs[i]
  
    for j in range(i+1,n):
      seqB = seqs[j]
      
      score, alignA, alignB = sequenceAlign(seqA, seqB, simMatrix)
      maxScore = max(maxScores[i],maxScores[j])
      dist = maxScore - score
      
      matrix[i][j] = dist
      matrix[j][i] = dist

  return matrix


In [30]:
def sequenceAlign(seqA, seqB, simMatrix=DNA_1, insert=8, extend=4):
    
  """Function to align two sequences.
  Takes as arguments two one-letter sequences, a substitution matrix and two gap penalties.
  Returns alignment score and aligned sequences."""


  numI = len(seqA) + 1
  numJ = len(seqB) + 1

  scoreMatrix = [[0] * numJ for x in range(numI)]
  routeMatrix = [[0] * numJ for x in range(numI)]
  
  for i in range(1, numI):
    routeMatrix[i][0] = 1
  
  for j in range(1, numJ):
    routeMatrix[0][j] = 2
  
  for i in range(1, numI):
    for j in range(1, numJ):
    
      penalty1 = insert
      penalty2 = insert
      
      if routeMatrix[i-1][j] == 1:
        penalty1 = extend
        
      elif routeMatrix[i][j-1] == 2:
        penalty2 = extend
        
      similarity = simMatrix[ seqA[i-1] ][ seqB[j-1] ]
      
      paths = [scoreMatrix[i-1][j-1] + similarity, # Route 0
               scoreMatrix[i-1][j] - penalty1, # Route 1
               scoreMatrix[i][j-1] - penalty2] # Route 2                     
      
      best = max(paths)
      route = paths.index(best)           

      scoreMatrix[i][j] = best
      routeMatrix[i][j] = route
      
  alignA = []
  alignB = []
  
  i = numI-1
  j = numJ-1
  score = scoreMatrix[i][j]

    
  while i > 0 or j > 0:
    route = routeMatrix[i][j]    

    if route == 0: # Diagonal
      alignA.append( seqA[i-1] )
      alignB.append( seqB[j-1] )
      i -= 1
      j -= 1

    elif route == 1: # Gap in seqB
      alignA.append( seqA[i-1] )
      alignB.append( '-' )
      i -= 1      

    elif route == 2: # Gap in seqA
      alignA.append( '-' )
      alignB.append( seqB[j-1] ) 
      j -= 1
  
  alignA.reverse()
  alignB.reverse()
  alignA = ''.join(alignA)
  alignB = ''.join(alignB)

  return score, alignA, alignB 

In [31]:
# calculate distance matrix of DNA alignments
distMatrix = getDistanceMatrix(align, DNA_1)

In [32]:
# print formatted distance matrix
for row in distMatrix:
    print(['%.2f' % x for x in row])

['0.00', '17.00', '17.00', '14.00', '16.00', '15.00', '16.00']
['17.00', '0.00', '13.00', '17.00', '19.00', '17.00', '15.00']
['17.00', '13.00', '0.00', '16.00', '18.00', '13.00', '15.00']
['14.00', '17.00', '16.00', '0.00', '15.00', '16.00', '15.00']
['16.00', '19.00', '18.00', '15.00', '0.00', '16.00', '15.00']
['15.00', '17.00', '13.00', '16.00', '16.00', '0.00', '12.00']
['16.00', '15.00', '15.00', '15.00', '15.00', '12.00', '0.00']
